In [ ]:
# Libraries

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sas
import xgboost as xgb
%matplotlib inline
plt.style.use('ggplot')
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
stop = set(stopwords.words('english'))
import os
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import socket
from urllib.request import urlopen
from PIL import Image
import time
import warnings
warnings.filterwarnings("ignore")#忽略掉普通的warning
print(os.listdir("../input"))

我是QQ756745172。这是的我第一个内核,我一直信奉一句话:天行健,君子自强不息

In [ ]:
train=pd.read_csv('../input/tmdb-box-office-prediction/train.csv',index_col=0)
test=pd.read_csv('../input/tmdb-box-office-prediction/test.csv',index_col=0)

In [ ]:
# trainAdditionalFeatures = pd.read_csv('../input/feature-2/TrainAdditionalFeatures.csv')[['imdb_id','popularity2','rating']]
# testAdditionalFeatures = pd.read_csv('../input/feature-2/TestAdditionalFeatures.csv')[['imdb_id','popularity2','rating']]

# train = pd.merge(train, trainAdditionalFeatures, how='left', on=['imdb_id'])
# test = pd.merge(test, testAdditionalFeatures, how='left', on=['imdb_id'])
# train['id']=train.index+1
# test['id']=test.index+1

In [ ]:
# release_dates = pd.read_csv('../input/feature-3/release_dates_per_country.csv')
# release_dates['id'] = range(1,7399)
# release_dates.drop(['original_title','title'],axis = 1,inplace = True)
# release_dates.index = release_dates['id']
# train = pd.merge(train, release_dates, how='left', on=['id'])
# test = pd.merge(test, release_dates, how='left', on=['id'])

In [ ]:
train.head()

In [ ]:
test['revenue']=-99

In [ ]:
test.head()

In [ ]:
train[['belongs_to_collection']].info()

In [ ]:
train[['belongs_to_collection','title']].head()

我只是分享我的工作历程,有啥不对的或者建议欢迎留言。

通过观察,这个应该是个干扰特征,看起来我们可以删除它

In [ ]:
train=train.drop('belongs_to_collection',axis=1)
test=test.drop('belongs_to_collection',axis=1)

以下数据调整来自内核,目的是为了修改不正确的信息

In [ ]:
train.loc[train.index == 16,'revenue'] = 192864         
train.loc[train.index == 90,'budget'] = 30000000                  
train.loc[train.index== 118,'budget'] = 60000000       
train.loc[train.index== 149,'budget'] = 18000000       
train.loc[train.index== 313,'revenue'] = 12000000       
train.loc[train.index == 451,'revenue'] = 12000000      
train.loc[train.index == 464,'budget'] = 20000000       
train.loc[train.index == 470,'budget'] = 13000000       
train.loc[train.index== 513,'budget'] = 930000         
train.loc[train.index == 797,'budget'] = 8000000        
train.loc[train.index == 819,'budget'] = 90000000       
train.loc[train.index == 850,'budget'] = 90000000       
train.loc[train.index == 1007,'budget'] = 2              
train.loc[train.index== 1112,'budget'] = 7500000       
train.loc[train.index == 1131,'budget'] = 4300000        
train.loc[train.index == 1359,'budget'] = 10000000       
train.loc[train.index == 1542,'budget'] = 1             
train.loc[train.index == 1570,'budget'] = 15800000       
train.loc[train.index== 1571,'budget'] = 4000000        
train.loc[train.index == 1714,'budget'] = 46000000       
train.loc[train.index == 1721,'budget'] = 17500000       
train.loc[train.index== 1865,'revenue'] = 25000000      
train.loc[train.index == 1885,'budget'] = 12             
train.loc[train.index == 2091,'budget'] = 10             
train.loc[train.index == 2268,'budget'] = 17500000       
train.loc[train.index == 2491,'budget'] = 6              
train.loc[train.index == 2602,'budget'] = 31000000       
train.loc[train.index == 2612,'budget'] = 15000000       
train.loc[train.index == 2696,'budget'] = 10000000      
train.loc[train.index == 2801,'budget'] = 10000000       
train.loc[train.index == 335,'budget'] = 2 
train.loc[train.index == 348,'budget'] = 12
train.loc[train.index == 470,'budget'] = 13000000 
train.loc[train.index == 513,'budget'] = 1100000
train.loc[train.index == 640,'budget'] = 6 
train.loc[train.index == 696,'budget'] = 1
train.loc[train.index == 797,'budget'] = 8000000 
train.loc[train.index == 850,'budget'] = 1500000
train.loc[train.index == 1199,'budget'] = 5 
train.loc[train.index == 1282,'budget'] = 9              
train.loc[train.index== 1347,'budget'] = 1
train.loc[train.index== 1755,'budget'] = 2
train.loc[train.index == 1801,'budget'] = 5
train.loc[train.index == 1918,'budget'] = 592 
train.loc[train.index == 2033,'budget'] = 4
train.loc[train.index == 2118,'budget'] = 344 
train.loc[train.index == 2252,'budget'] = 130
train.loc[train.index == 2256,'budget'] = 1 
train.loc[train.index == 2696,'budget'] = 10000000

In [ ]:
test.loc[test.index== 3033,'budget'] = 250 
test.loc[test.index== 3051,'budget'] = 50
test.loc[test.index == 3084,'budget'] = 337
test.loc[test.index == 3224,'budget'] = 4  
test.loc[test.index == 3594,'budget'] = 25  
test.loc[test.index == 3619,'budget'] = 500  
test.loc[test.index == 3831,'budget'] = 3  
test.loc[test.index== 3935,'budget'] = 500  
test.loc[test.index == 4049,'budget'] = 995946 
test.loc[test.index== 4424,'budget'] = 3  
test.loc[test.index == 4460,'budget'] = 8  
test.loc[test.index == 4555,'budget'] = 1200000 
test.loc[test.index== 4624,'budget'] = 30 
test.loc[test.index== 4645,'budget'] = 500 
test.loc[test.index == 4709,'budget'] = 450 
test.loc[test.index == 4839,'budget'] = 7
test.loc[test.index== 3125,'budget'] = 25 
test.loc[test.index== 3142,'budget'] = 1
test.loc[test.index == 3201,'budget'] = 450
test.loc[test.index == 3222,'budget'] = 6
test.loc[test.index== 3545,'budget'] = 38
test.loc[test.index == 3670,'budget'] = 18
test.loc[test.index == 3792,'budget'] = 19
test.loc[test.index == 3881,'budget'] = 7
test.loc[test.index == 3969,'budget'] = 400
test.loc[test.index == 4196,'budget'] = 6
test.loc[test.index == 4221,'budget'] = 11
test.loc[test.index == 4222,'budget'] = 500
test.loc[test.index== 4285,'budget'] = 11
test.loc[test.index == 4319,'budget'] = 1
test.loc[test.index == 4639,'budget'] = 10
test.loc[test.index == 4719,'budget'] = 45
test.loc[test.index == 4822,'budget'] = 22
test.loc[test.index == 4829,'budget'] = 20
test.loc[test.index== 4969,'budget'] = 20
test.loc[test.index== 5021,'budget'] = 40 
test.loc[test.index== 5035,'budget'] = 1 
test.loc[test.index== 5063,'budget'] = 14 
test.loc[test.index == 5119,'budget'] = 2 
test.loc[test.index== 5214,'budget'] = 30 
test.loc[test.index== 5221,'budget'] = 50 
test.loc[test.index== 4903,'budget'] = 15
test.loc[test.index == 4983,'budget'] = 3
test.loc[test.index == 5102,'budget'] = 28
test.loc[test.index== 5217,'budget'] = 75
test.loc[test.index == 5224,'budget'] = 3 
test.loc[test.index== 5469,'budget'] = 20 
test.loc[test.index == 5840,'budget'] = 1 
test.loc[test.index == 5960,'budget'] = 30
test.loc[test.index == 6506,'budget'] = 11 
test.loc[test.index== 6553,'budget'] = 280
test.loc[test.index == 6561,'budget'] = 7
test.loc[test.index== 6582,'budget'] = 218
test.loc[test.index == 6638,'budget'] = 5
test.loc[test.index== 6749,'budget'] = 8 
test.loc[test.index==6759,'budget'] = 50 
test.loc[test.index == 6856,'budget'] = 10
test.loc[test.index== 6858,'budget'] =  100
test.loc[test.index == 6876,'budget'] =  250
test.loc[test.index == 6972,'budget'] = 1
test.loc[test.index== 7079,'budget'] = 8000000
test.loc[test.index == 7150,'budget'] = 118
test.loc[test.index == 6506,'budget'] = 118
test.loc[test.index == 7225,'budget'] = 6
test.loc[test.index == 7231,'budget'] = 85
test.loc[test.index == 5222,'budget'] = 5
test.loc[test.index == 5322,'budget'] = 90
test.loc[test.index == 5350,'budget'] = 70
test.loc[test.index == 5378,'budget'] = 10
test.loc[test.index== 5545,'budget'] = 80
test.loc[test.index == 5810,'budget'] = 8
test.loc[test.index== 5926,'budget'] = 300
test.loc[test.index== 5927,'budget'] = 4
test.loc[test.index== 5986,'budget'] = 1
test.loc[test.index == 6053,'budget'] = 20
test.loc[test.index== 6104,'budget'] = 1
test.loc[test.index == 6130,'budget'] = 30
test.loc[test.index == 6301,'budget'] = 150
test.loc[test.index == 6276,'budget'] = 100
test.loc[test.index == 6473,'budget'] = 100
test.loc[test.index== 6842,'budget'] = 30

In [ ]:
new_data=pd.concat([train,test],axis=0)
new_data.head()

In [ ]:
new_data.shape

合并数据集进行数据清洗和关键信息提取.内核上多为循环,我使用的是正则表达式。

In [ ]:
new_data.isnull().sum()

上映时间有空值,我们可以把它查出来,填上 

In [ ]:
new_data[new_data['release_date'].isnull()]

In [ ]:
new_data['release_date']=new_data['release_date'].fillna('3/20/01')

Now let's deal with the release column.

In [ ]:
# new_data['release_year']=pd.to_datetime(new_data['release_date']).dt.year
new_data['release_month']=pd.to_datetime(new_data['release_date']).dt.month
new_data['release_day']=pd.to_datetime(new_data['release_date']).dt.day

把字符时间分割为可使用的数字化时间

In [ ]:
new_data['release_year'].loc[new_data['release_year']>=2018]-=100

In [ ]:
new_data[['release_date','release_year','release_month','release_day']].head()

In [ ]:
new_data=new_data.drop('release_date',axis=1)

In [ ]:
new_data['homepage_fact']=new_data['homepage'].apply(lambda x: 0 if x is np.nan  else 1)

In [ ]:
a=new_data.loc[train.index].loc[train.index].groupby('homepage_fact').revenue.mean()
plt.bar(a.index,a.values)

有网页的比没网页的电影平均票房要高出一倍不止,说明有一个网页对电影的票房有很大影响

个人通过类比法,通过车牌与人之间的关系,域名可能隐藏着公司的强弱关系,所以需要网页进行域名抽取

In [ ]:
new_data['homepage_end']=new_data[new_data['homepage'].notna()]['homepage'].str.findall(r'\.([a-z]+)(?:\/|$)').apply(lambda x:x[0])
new_data['homepage_end'].head()

In [ ]:
new_data['homepage_end']=new_data['homepage_end'].fillna('unknow')

In [ ]:
# new_data=new_data.drop('id',axis=1)

In [ ]:
new_data['homepage_end']=new_data.update(new_data.loc[train.index].groupby('homepage_end').revenue.median().sort_values(ascending=False))
plt.figure(figsize=(20,10))
sas.barplot(a.index,a)

通过域名后缀聚合票房,观察到国际域名后缀的网站带来的票房收益平均值更大,甚至看起来超过了其他域名的累加和

In [ ]:
page=pd.get_dummies(new_data['homepage_end'])
page.head()

由于域名不具备可排序性,所以需要用one-hot编码,避免数字特征的大小影响模型效果

In [ ]:
new_data=new_data.drop('homepage',axis=1)

In [ ]:
new_data['poster_path'].describe()

电影的海报,在预测上肯定是用不到了,不过用来做可视化挺好的,后面我将会用原始数据进行操作,所以在新数据这里我打算删除它

In [ ]:
new_data=new_data.drop('poster_path',axis=1)

In [ ]:
new_data['len_overview']=new_data['overview'].fillna('NAN').apply(lambda x:len(x))

电影的简介长度也可能影响电影票房,过短可能无法提起用户兴趣,过长可能会让用户没有耐心

In [ ]:
new_data.plot(x="len_overview",y="revenue", kind="scatter",figsize=(12,8))

这样子是不是什么都看不出来？由于电影简介长度没有统一标准,所以我需要分位数来离散化数据,使数据更具有表示性

In [ ]:
len_rew_sort=new_data['len_overview'].sort_values(ascending=True)
len_rew_sort.head()

先试试十分位把

In [ ]:
length=len(new_data['len_overview'])
m=0.1
n=0.1
arr_len_ove=[]
for i in range(1,11):
    arr_len_ove.append(round(length*m))
    m+=n
arr_len_ove

计算位置

In [ ]:
exam=len_rew_sort.iloc[1480:2959].value_counts()
sas.barplot(exam.index,exam.values)

数据没有出现偏尾分布的情况,所以我采取均值用来衡量这段区间的一个整体分布

In [ ]:
qu_arr=[]
for i in range(10):
    if i==0:
            x=len_rew_sort.iloc[:arr_len_ove[0]].mean()
    else:
        x=len_rew_sort.iloc[arr_len_ove[i-1]:arr_len_ove[i]].mean()
    qu_arr.append(round(x))

In [ ]:
for i in range(10):
    qu=qu_arr[i]
    if i==0:
        new_data['len_overview'].loc[(new_data['len_overview']<len_rew_sort.iloc[arr_len_ove[i]-1])]=qu
    else:
        new_data['len_overview'].loc[(new_data['len_overview']<len_rew_sort.iloc[arr_len_ove[i]-1])&(new_data['len_overview'] >qu_arr[i-1])]=qu
    print(i,qu)

In [ ]:
np.sort(new_data['len_overview'].unique())

In [ ]:
len_ove_agg=new_data.groupby('len_overview').revenue.aggregate(['min','max','std'])

In [ ]:
len_ove_agg.plot()

从分析上来看,票房的方差和最小值受字数影响不大,但是最大值,字符长度为95-500前看起来像一个平稳序列。并且500以后呈现出明显的下降趋势。所以电影简介的字符长度应该控制在450以内

In [ ]:
new_data.head()

In my opinion, the cost of making movies should be divided into millions and billions. 

In [ ]:
sas.relplot('budget','revenue',data=train)

票房和预算一定程度上正相关

In [ ]:
new_data['geres_name']=new_data['genres'].str.findall(r'\'name\'\s?:\s?\'(\w+)\'')
new_data['geres_name'].head()

In [ ]:
new_data=new_data.drop('genres',axis=1)

In [ ]:
country=new_data['production_countries'].str.findall(r'[A-Z]{2,5}')


In [ ]:
new_data['production_countries']=country


In [ ]:
new_data['original_and_new']=new_data['original_title']==new_data['title']

In [ ]:
new_data=new_data.drop('original_title',axis=1)
# new_data=new_data.drop('title',axis=1)
new_data.head()

In [ ]:
new_data=pd.concat([new_data,page],axis=1)

In [ ]:
new_data['production_companies']=new_data['production_companies'].str.findall(r'\'name\'?:\s?\'([A-Za-z]+)')
new_data.fillna('Unknow')
print('接下来就是地图可视化了')

In [ ]:
new_data['imdb_id'].describe()

这个特征没啥用,因为其他特征也存在这个特征

In [ ]:
new_data=new_data.drop('imdb_id',axis=1)

In [ ]:
new_data['spoken_languages']=new_data['spoken_languages'].str.findall(r'\'([a-z]{2})\'')

In [ ]:
new_data['production_companies']=new_data['production_companies'].fillna('unknow')

In [ ]:
new_data['spoken_languages']=new_data['spoken_languages'].fillna('unknow')

提取电影语言信息

In [ ]:
new_data['Keywords']=new_data['Keywords'].str.findall(r'\'?:\s?\'([a-z]+\s?[a-z]+)\'').fillna('unkonw')

In [ ]:
new_data['Keywords'][1]

In [ ]:
m=list(new_data['cast'].str.findall(r'\'name\'?:\s?\'(\S+\s?\S+)\'').fillna('unknow'))

In [ ]:
plt.figure(figsize = (16, 12))
list_of_keywords=list(new_data['Keywords'].loc[new_data['Keywords']!='unkonw'])
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_keywords for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='Top keywords'
plt.axis("off")
plt.show()

词云

In [ ]:
d = Counter([i for j in list_of_keywords for i in j]).most_common(30)
d

In [ ]:
for i in d:
    m=i[0]
    new_data['Keywords_'+m]=new_data['Keywords'].apply(lambda x:1 if m in x else 0)

In [ ]:
new_data=new_data.drop('Keywords',axis=1)

In [ ]:
new_data['cast']=new_data['cast'].fillna('unknow')

In [ ]:
m=list(new_data['cast'].str.replace(r'\s+\'order\':\s?\d+\S?\s\'profile_path\'?:\s?\'','').str.findall('\'name\'?:\s?\'(\D+)\'\S?\s?\'?(\/\w+.jpg)\'?\}'))
d = Counter([i for j in m for i in j]).most_common(16)
d

In [ ]:
fig = plt.figure(figsize=(20, 12))
for i,p in enumerate([j[0] for j in d]):
    p=str(p).split(',')
    m=p[0][2:]
    m=m[:len(m)-1]
    p=p[1]
    p=p[2:len(p)-2]
    print("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p)
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(urlopen("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p))         
    plt.imshow(im)
    ax.set_title(m)
    

绘制著名演员

In [ ]:
for i in d:
    m=i[0][0]
    new_data['cast_name_'+m]=new_data['cast'].apply(lambda x:1 if m in x else 0)

判断著名演员是否在里面,因为著名演员会在一定程度上影响票房

In [ ]:
new_data['cast'][1]

In [ ]:
list_cast_gender=list(new_data['cast'].str.findall(r'\'gender\'\s?:\s?(\d+)\s?'))
Counter([i for j in list_cast_gender for i in j]).most_common()

统计男女占比

In [ ]:
new_data['cast_gender_0']=new_data['cast'].str.findall(r'\'gender\'\s?:\s?(\d+)\s?').apply(lambda x: x.count('0'))
new_data['cast_gender_1']=new_data['cast'].str.findall(r'\'gender\'\s?:\s?(\d+)\s?').apply(lambda x: x.count('1'))
new_data['cast_gender_2']=new_data['cast'].str.findall(r'\'gender\'\s?:\s?(\d+)\s?').apply(lambda x: x.count('2'))

In [ ]:
list_cast_char=list(new_data['cast'].str.findall(r'\'character\'\s?:\s?\'(\S+)\'\S?\s?'))
top_cast_char=Counter([i for j in list_cast_char for i in j]).most_common(20)

In [ ]:
for g in top_cast_char:
    m=g[0]
    new_data['cast_char_'+m]=new_data['cast'].apply(lambda x:1 if m in x else 0)

In [ ]:
new_data=new_data.drop('cast',axis=1)

In [ ]:
new_data['status']=new_data['status'].fillna('Unknow')

In [ ]:
new_data['overview']=new_data['overview'].fillna('')

In [ ]:
plt.figure(figsize = (12, 12))
text = ' '.join(train['original_title'].values)
wordcloud = WordCloud(max_font_size=None, background_color='white', width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='Top words in titles'
plt.axis("off")
plt.show()

In [ ]:
plt.figure(figsize = (12, 12))
text = ' '.join(train['overview'].fillna('').values)
wordcloud = WordCloud(max_font_size=None, background_color='white', width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='Top words in overview'
plt.axis("off")
plt.show()

In [ ]:
from sklearn.linear_model import  LinearRegression
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


In [ ]:
log_re=np.log1p(train['revenue'])
v1=TfidfVectorizer( sublinear_tf=True,analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1, 2),
            min_df=5)
overview_text = v1.fit_transform(train['overview'].fillna(''))
linre=LinearRegression()
linre.fit(overview_text,log_re)


In [ ]:
eli5.show_weights(linre, vec=v1, top=20, feature_filter=lambda x: x !='<BIAS>')

根据权重排序

In [ ]:
print('Target value:',log_re[1000])
eli5.show_prediction(linre, doc=train['overview'].values[1000], vec=v1)


In [ ]:
plt.figure(figsize = (12, 12))
text_ta=' '.join(new_data['tagline'].fillna('').values)
wordcloud=WordCloud(max_font_size=None,background_color='white',width=1200,height=1000).generate(text_ta)
plt.imshow(wordcloud)
plt.title='Top words in tagline'
plt.axis("off")
plt.show()

In [ ]:

# new_data['crew']=new_data['crew'].str.replace(',','').str.replace('}','').str.replace(r'\S+credit_id\S+:\s?\s?\S?\'(\S+)\S+\'','').str.findall(r'\'job\S?\'\s?:\s?\'(\S+\s?\S+)\S\s?\S?\'name\S?\'\s?:\s?\'(\S+\s?\S+)\'\s?\S?\'profile_path\S?\':\s+\'?(\S+)\S?\s?\s?\S?department\S?\'?:\s?\S?(\S+)\s?\S?\'?') 

In [ ]:
new_data['crew_0']=new_data['crew'].fillna('').str.replace(',','').str.replace('}','').str.findall('\'gender\S?\'\s?:\s?\S?(\d+)\s?\S?').apply(lambda x:x.count('0'))
new_data['crew_1']=new_data['crew'].fillna('').str.replace(',','').str.replace('}','').str.findall('\'gender\S?\'\s?:\s?\S?(\d+)\s?\S?').apply(lambda x:x.count('1'))
new_data['crew_2']=new_data['crew'].fillna('').str.replace(',','').str.replace('}','').str.findall('\'gender\S?\'\s?:\s?\S?(\d+)\s?\S?').apply(lambda x:x.count('2'))

In [ ]:
list_crew_depart=list(new_data['crew'].fillna('').str.replace(',','').str.replace('}','').str.findall('\'department\'\s?\S?:\s+\S?(\S+)\S?\s?\S?'))
d=Counter([i for j in list_crew_depart for i in j]).most_common(15)

In [ ]:
list_crew_job=list(new_data['crew'].fillna('').str.replace(',','').str.replace('}','').str.findall('\'job\S?\'\s?:\s?\'(\D+)\S\s?\S?\'name'))
dd=Counter([i for j in list_crew_job for i in j]).most_common(15)

In [ ]:
for i in d:
    new_data['crew_depart_is_'+i[0]]=new_data['crew'].fillna('').apply(lambda x:1 if i[0] in x else 0)
for i in dd:
    new_data['crew_job_is_'+i[0]]=new_data['crew'].fillna('').apply(lambda x:1 if i[0] in x else 0)

In [ ]:
new_data['crew']=new_data['crew'].str.replace(',','').str.replace('}','').str.replace(r'\'gender\'\s?\S?:\s?\S?\s+\'id\'\s?:\s?\d+','').str.findall(r'\'department\'\s?\S?:\s+\S?(\S+)\S?\s?\S?\s+\'job\S?\'\s?:\s?\'(\D+)\S\s?\S?\'name\S?\'\s?:\s?\'(\D+)\'\s?\S?\'profile_path\S?\':\s+\'?(\S+)\S?\s?\s?\S?') 

此段代码是为了提取每一个电影参与制作的人员姓名,海报,工作和部门,下面将会展示处理和没处理的情况

In [ ]:
new_data['len_crew']=new_data['crew'].fillna('1').apply(lambda x:len(x))

电影制作人员数量也可以当作电音票房预测的特征

In [ ]:
new_data['crew'][1][:5]

为了美观,只显示前五个,原始数据如下面所示

In [ ]:
train['crew'][1]

In [ ]:
# new_data['crew'].str.replace(',','').str.replace('}','').str.replace(r'\S+credit_id\S+:\s?\s?\S?\'(\S+)\S+\'','').str.findall(r'\'job\S?\'\s?:\s?\'(\S+\s?\S+)\S\s?\S?\'name\S?\'\s?:\s?\'(\S+\s?\S+)\'\s?\S?\'profile_path\S?\':\s+\S?(\S+)\S?\s?\s?\S?department\S?\'?:\s?\S?(\S+)\s?\S?')[1] 

In [ ]:
# train['crew'].str.replace(',','').str.replace('}','').str.replace(r'\S+credit_id\S+:\s?\s?\S?\'(\S+)\S+\'','').str.findall(r'\'job\S?\'\s?:\s?\'(\S+\s?\S+)\S\s?\S?\'name\S?\'\s?:\s?\'(\S+\s?\S+)\'\s?\S?\'profile_path\S?\':\s+\'?(\S+)\S?\s?\s?\S?department\S?\'?:\s?\S?(\S+)\s?\S?')[1] 

In [ ]:
# new_data=new_data.drop('crew',axis=1)

In [ ]:
new_train=new_data.loc[np.array(train.index)]

分析制造国家与票房之间的联系

In [ ]:
new_train['production_countries']=new_train['production_countries'].fillna('')

In [ ]:
new_train['production_countries']=new_train['production_countries'].apply(lambda x:'_'.join(x))

In [ ]:
count=new_train['production_countries'].value_counts()
count=count[count>5]

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(count.index,count.values)
plt.show()

美国在电影制作上产量很大

In [ ]:
bud=new_train.groupby('production_countries').budget.mean()

In [ ]:
bud=bud.sort_values(ascending=False)[:10]

In [ ]:
plt.figure(figsize=(20,12))
plt.bar(bud.index,bud.values)
plt.show()

在电影方面只要是有美国参与的情况下,电影的投资预算总是差不多的,并且远高于其他国家


In [ ]:
rev=new_train.groupby('production_countries').revenue.mean()

In [ ]:
rev=rev.sort_values(ascending=False)[:10]

In [ ]:
plt.figure(figsize=(20,12))
plt.bar(rev.index,rev.values)
plt.show()

投入与产出并不成正比，cz_us_AE这三个国家的收益远高于其他的国家或者合作国家

In [ ]:
new_train['release_month'].unique()

In [ ]:
new_train['production_countries'].loc[new_train['production_countries']=='ET']='Ethiopia'
along_co=new_train[new_train['production_countries'].apply(lambda x:1 if len(x)==2 else 0)==1]
along_co.head()

In [ ]:
count_rev=along_co[['production_countries','revenue']]

In [ ]:
dd=count_rev['production_countries'].unique()
mn=pd.DataFrame(dd,columns=['address'])
mn.head()

In [ ]:
import geopandas as ge
# boros = ge.read_file(ge.datasets.get_path("nybb"))
# m1=ge.tools.geocode(count_rev['production_countries'].unique(), provider='nominatim', user_agent="my-application")

搜集国家地图中心坐标,可以用来做简易的几何可视化,后面所做的可视化是用板块可视化的

In [ ]:
# mn=m1.copy()
# mn

In [ ]:
world = ge.read_file(ge.datasets.get_path('naturalearth_lowres'))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
dd=count_rev['production_countries'].value_counts()
mn['values']=dd.values
mn.head()

In [ ]:
mn['address'][36]

In [ ]:
mn['address'][0]='United States'
mn['address'][2]='Korea'
mn['address'][1]='India'
mn['address'][3]='Serbia'
mn['address'][4]='United Kingdom'
mn['address'][5]='France'
mn['address'][6]='New Zealand'
mn['address'][7]='Italy'
mn['address'][8]='Belgium'
mn['address'][9]='Czech Rep.'
mn['address'][11]='Russia'
mn['address'][12]='Spain'
mn['address'][13]='Turkey'
mn['address'][14]='China'
mn['address'][15]='Canada'
mn['address'][16]='Australia'
mn['address'][17]='Iran'
mn['address'][18]='Japan'
mn['address'][19]='Sweden'
mn['address'][20]='Philippines'
mn['address'][21]='Brazil'
mn['address'][22]='Netherlands'
mn['address'][23]='Ireland'
mn['address'][24]='Mexico'
mn['address'][25]='Germany'
mn['address'][26]='South Africa'
mn['address'][27]='Finland'
mn['address'][28]='Pakistan'
mn['address'][29]='Norway'
mn['address'][30]='Bulgaria'
mn['address'][31]='Ukraine'
mn['address'][32]='Denmark'
mn['address'][33]='Romania'
mn['address'][35]='Hungary'
mn['address'][36]='Chile'
mn['address'][37]='Indonesia'
mn['address'][38]='Poland'


由于电影数据库是二字母缩写,但是geopandas只有三字母缩写，所以我必须通过查询来修正,但是有二字母和三字母缩写的数据集就好办了

In [ ]:
mn=mn.drop(10)
mn=mn.drop(34)

geopandas无法同时绘制点和区域,地理特征数据类型必须一致

In [ ]:
mn['geometry']='unknow'

In [ ]:
for i in range(39):
    if i==10 or i==34:
        continue;
    d=world[world['name']==mn['address'][i]]['geometry'].values[0]
    mn['geometry'][i]=d

现在我们已经有了板块信息了

In [ ]:
fig, ax = plt.subplots()
mn=ge.GeoDataFrame(mn)
ax.set_aspect('equal')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
world.plot(ax=ax, color='white', edgecolor='black')
mn.plot(column='values',ax=ax, legend=True,cmap=plt.get_cmap('rainbow'))


通过geopandas的地图映射方法可视化国家与影片数量的关系

In [ ]:
TS = train.loc[:,["original_title","release_date","budget","runtime","revenue"]]
TS.dropna()

TS.release_date = pd.to_datetime(TS.release_date)
TS.loc[:,"Year"] = TS["release_date"].dt.year
TS.loc[:,"Month"] = TS["release_date"].dt.month
TS = TS[TS.Year<2018]

接下来,让我们做一些有趣的可视化

In [ ]:
titles = TS.groupby("Year")["original_title"].count()
titles.plot(figsize=(15,8))
plt.xlabel="Year of release"
plt.ylabel="Number of titles released"
plt.xticks(np.arange(1970,2025,5))
plt.show()

电影每年的产量都在增加

In [ ]:
rev=TS.groupby('Year')['revenue'].aggregate(['min','mean','max','std'])
rev.plot(figsize=(20,10))
plt.xlabel="Year of release"
plt.ylabel="Revenue"
plt.xticks(np.arange(1970,2025,5))
plt.show()

 每年电影的产出最大值在不断增长,有一个很明显的趋势性,

In [ ]:
bud=TS.groupby('Year')['budget'].aggregate(['min','max','mean','std'])
bud.plot(figsize=(20,10))
plt.xlabel="Year of release"
plt.ylabel="Budget"
plt.xticks(np.arange(1970,2025,5))
plt.show()

电影预算每年的最大值也在不断攀升,说明从95年开始,全世界的电影行业在不断飞速发展

In [ ]:
TS[TS['budget']==0].head()

很明显,通过互联网查询,这个预算为0的是缺失值

In [ ]:
runtimes = TS.groupby("Year")["runtime"].aggregate(["min","mean","max"])
runtimes.plot(figsize=(15,8))
plt.xlabel="Year of release"
plt.ylabel="Runtime"
plt.xticks(np.arange(1970,2025,5))
plt.show()

In [ ]:
r_zeros = TS[TS.runtime==0]
r_zeros.head()

很明显,这个也属于缺失值

In [ ]:
train.plot(x="runtime",y="budget", kind="scatter",figsize=(12,8))
plt.show()

电影的时间和预算没有相关性

In [ ]:
train.plot(x="runtime",y="revenue", kind="scatter",figsize=(12,8))
plt.show()

电影播放时长在80-150分钟的可以获得一个更稳定更好的票房

In [ ]:
train.plot(x="popularity",y="budget", kind="scatter",figsize=(12,8))
plt.show()

让我们除去那些特别知名的电影干扰,

In [ ]:
pop = train[train.popularity<50]
pop=pop[pop['budget']!=0]
pop.plot(x="budget",y="popularity", kind="scatter",figsize=(12,8))
plt.show()

预算和流行程度没什么关系,增加成本也可能无法获得足够的名气

In [ ]:
top3 = train.sort_values(by='popularity',ascending=False)[:10]
id3=top3[['title','poster_path','revenue']]

In [ ]:
fig = plt.figure(figsize=(20, 12))
cont=0
for i in id3.index:
    p=id3.loc[i]
    ax = fig.add_subplot(4, 4, cont+1, xticks=[], yticks=[])
    print("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p['poster_path'])
    im = Image.open(urlopen("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p['poster_path']))
    plt.imshow(im)
    ax.set_title(p['title'])
    cont+=1

数据挖掘出来的流行电影排行榜

In [ ]:
old_title=new_data['title'].value_counts()

In [ ]:
a=old_title[old_title.values==3].index
b=old_title[old_title==2].index

In [ ]:
new_data['fan_pai_2']=new_data['title'].apply(lambda x:1 if x in a else 0)
new_data['pan_pai_3']=new_data['title'].apply(lambda x:1 if x in b else 0)

处理完翻拍的电影以后我们处理高票房电影是否存在续集,高票房在整个票房预测中对方差影响最大

In [ ]:
new_data.loc[new_data['title'].fillna('un').str.contains('Planet of the Apes')]

In [ ]:
top10 = train.sort_values(by='revenue',ascending=False)[:10]
id10=top10[['title','poster_path','revenue']]

In [ ]:
fig = plt.figure(figsize=(20, 12))
cont=0
for i in id10.index:
    p=id10.loc[i]
    ax = fig.add_subplot(4, 4, cont+1, xticks=[], yticks=[])
    print("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p['poster_path'])
    im = Image.open(urlopen("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p['poster_path']))
    plt.imshow(im)
    ax.set_title(p['title'])
    cont+=1

电影票房排行榜

将近48小时的劳动成果,还有很多分析没做完。

In [ ]:
title_count=train['title'].value_counts()
len(title_count[title_count.values!=1])

31部电影是经过翻拍的

In [ ]:
train[train['title'].str.contains('Furious')]['title']


In [ ]:
test[test['title'].fillna('Unknow').str.contains('Furious')]['title']

高票房电影很大概率为系列电影,比如速度与激情,并且它的系列电影呗分割在训练集和测试集中

In [ ]:
list_crew=list(new_data['crew'].fillna('Unknow'))
d = Counter([j for i in list_crew for j in i]).most_common(17)

In [ ]:
d.remove(d[7])

In [ ]:
for i in d :
    new_data['crew_contains_'+i[0][2]]=new_data['crew'].fillna('').apply(lambda x:1 if i[0][2] in x else 0)


In [ ]:
fig = plt.figure(figsize=(20, 16))
for i in range(16):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    p=d[i][0][3]
    p=p[:len(p)-1]
    print("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p)
    if p!='Non':
        im = Image.open(urlopen("https://image.tmdb.org/t/p/w600_and_h900_bestv2%s"%p))
    else:
        im = Image.new('RGB', (5, 5))
    plt.imshow(im)
    ax.set_title(' \n Name: %s\n Job:%s'%(d[i][0][2],d[i][0][1]))

In [ ]:
ts=new_train.groupby('release_month').revenue.median()

In [ ]:
plt.figure(figsize=(10,5))
sas.pointplot(ts.index,ts.values,ylabel='revenue')

我使用中位数排除了极值的干扰,通过数据分析暑假和寒假明显是电影票房的黄金档期,在秋季开学时间,电影票房最低

In [ ]:
mon_is=[1,6,7,9,10,12]
for i in mon_is:
    new_data['mon_is_'+str(i)]=new_data['release_month'].apply(lambda x:1 if x==i else 0)

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(ts.rolling(window=2,center=False).mean(),label='Rolling Mean');
# plt.plot(ts.rolling(window=2,center=False).std(),label='Rolling sd');
plt.plot(ts.rolling(window=3,center=False).mean(),label='Rolling Mean3')
plt.plot(ts.rolling(window=6,center=False).mean(),label='Rolling Mean6')
plt.legend();

电影的上半年和下半年票房差不多,但是下半年的波动性更明显,所以把电影安排在上半年上映效果比下半年更佳

In [ ]:
plt.plot(ts.diff(2))
plt.show()

好像票房与某几个月相关性太强,无法通过差分的方法把他变成平稳序列

In [ ]:
tt=TS.groupby('Year').revenue.median()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(tt.index,tt.values)
plt.xlabel='year'
plt.ylabel='revenue'
plt.title='revenue_of_year'
plt.show()

让我们试试差分法

In [ ]:
tt1=tt.diff(1)
plt.figure(figsize=(15,8))
plt.plot(tt1.index,tt1.values)
plt.xlabel='year'
plt.ylabel='revenue'
plt.title='revenue_of_year'
plt.show()

> > 序列看起来已经平稳了

In [ ]:
new_data['spoken_languages_count']=new_data['spoken_languages'].apply(lambda x:len(x))

In [ ]:
new_data.loc[train.index].groupby('spoken_languages_count').revenue.median()

In [ ]:
status_du=pd.get_dummies(new_data['status'])

In [ ]:
new_data=pd.concat([new_data,status_du],axis=1)


In [ ]:
new_data=new_data.drop(['status'],axis=1)
new_data.head()

In [ ]:
new_data=new_data.drop(['original_language'],axis=1)

In [ ]:
sum(new_data['geres_name'].isna())

这里有23个缺失值,我觉得可以通过查询来填充,但是暂时还是先用'unknow来代替

In [ ]:
new_data['geres_name']=new_data['geres_name'].fillna('Unknow')

In [ ]:
new_data['geres_name_count']=new_data['geres_name'].apply(lambda x:len(x))

In [ ]:
ge_name_count_rev=new_data.loc[train.index].groupby(['geres_name_count']).revenue.median().sort_values()

In [ ]:
sas.barplot(ge_name_count_rev.index,ge_name_count_rev.values)

影片元素越多,票房越高,当然貌似6种元素可能有点让人意外,毕竟我统计的是中位数,要排除票房的异常值的影响

In [ ]:
new_data=new_data.drop('homepage_end',axis=1)

In [ ]:
new_data['original_and_new']=new_data['original_and_new'].apply(lambda x: 1 if x else 0)

In [ ]:
plt.figure(figsize = (16, 12))
list_of_keywords=list(new_data['geres_name'])
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_keywords for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='Top geres_name'
plt.axis("off")
plt.show()

现在 我们可以发现 Comedy Thriller  Action  Drama Romance

In [ ]:
list_geres_name=['Comedy','Thriller','Action','Drama','Romance']

In [ ]:
for i in list_geres_name:
    new_data['geres_name'+'_'+i]=new_data['geres_name'].apply(lambda x: 1 if i in x else 0 )

现在,我们把排名前五的做一个特征,有利于算法预测的更精准

In [ ]:
new_data=new_data.drop('geres_name',axis=1)

In [ ]:
new_data['production_countries']=new_data['production_countries'].fillna('QQ')

In [ ]:
plt.figure(figsize = (16, 12))
list_of_keywords=list(new_data['production_countries'])
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_keywords for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='production_countries'
plt.axis("off")
plt.show()

In [ ]:
new_data['production_countries'].loc[new_data['production_countries']=='Unknow']['production_countries']='QQ'

In [ ]:
list_pro_coun=list(new_data['production_countries'])
d = Counter([j for i in list_pro_coun for j in i]).most_common(5)
d

看起来词云很能体现文本特征在总的集合里的数量比重

In [ ]:
for i in d:
     new_data['production_cou_name'+'_'+i[0]]=new_data['production_countries'].apply(lambda x: 1 if i[0] in x else 0 )

In [ ]:
new_data['pro_country_count']=new_data['production_companies'].apply(lambda x:len(x))

In [ ]:
pro_count_rev=new_data.loc[train.index].groupby('pro_country_count').revenue.median()

In [ ]:
sas.barplot(pro_count_rev.index,pro_count_rev.values)

数据里居然存在着16个国家合作的电影,但是好像在10和12个国家的制作的时候票房收益最高。当然我们也需要对他做一个one-hot当然你也可以选择离散化。我觉的one-hot更保险一些

In [ ]:
new_data=new_data.drop('production_countries',axis=1)

In [ ]:
new_data['production_companies'].loc[new_data['production_companies']=='unknow']='u'

In [ ]:
plt.figure(figsize = (16, 12))
list_of_pro_com=list(new_data['production_companies'])
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_pro_com for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='production_countries'
plt.axis("off")
plt.show()

好像巨头公司挺多的

In [ ]:
d = Counter([j for i in list_of_pro_com for j in i]).most_common(11)
d

我不应该把None一起拿进来计算的

In [ ]:
d.remove(('u',414))


制作电影数量排名前10的公司,

In [ ]:
dd=[]
for i in d:
    dd.append(i[0])

In [ ]:
fig = plt.figure(figsize=(20, 16))

for i in range(len(d)):
    m=d[i][0]
    new_data['is_'+m]=new_data['production_companies'].apply(lambda x:1 if m in x else 0)
    com=new_data[new_data['is_'+m]==1]
    val=com.groupby('release_year').budget.median()
    cou=com.release_year.value_counts()
    plt.plot(val.index,val.values)
    plt.legend(dd)  

当然,也可以选择画子图的方法,但是在这里不这么麻烦了

可能是比赛方防止过拟合,把一部分数据集的预算变成了0,我不知道填充了后结果是否会使算法表现的更好,所以,我并不打算修正这个错误

In [ ]:
plt.figure(figsize = (16, 12))
list_of_spk_lag=list(new_data['spoken_languages'].loc[new_data['spoken_languages']!='unknow'])
text = ' '.join(['_'.join(i.split(' ')) for j in list_of_spk_lag for i in j])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
plt.imshow(wordcloud)
plt.title='spk_lag'
plt.axis("off")
plt.show()

In [ ]:
d = Counter([j for i in list_of_spk_lag for j in i]).most_common(5)
d

In [ ]:
dd=[]
for i in d:
    dd.append(i[0])

In [ ]:
fig = plt.figure(figsize=(20, 16))

for i in range(len(d)):
    m=d[i][0]
    new_data['is_'+m]=new_data['spoken_languages'].apply(lambda x:1 if m in x else 0)
    com=new_data[new_data['is_'+m]==1]
    val=com.groupby('release_year').budget.median()
    cou=com.release_year.value_counts()
    plt.plot(val.index,val.values)
    plt.legend(dd)  
    plt.title='psk_release_median'

法国电影以前好像有过黄金期,英文电影预算变化相对于其他的看起来比较平稳

In [ ]:
new_data=new_data.drop('spoken_languages',axis=1)

In [ ]:
new_data.head()

In [ ]:
new_data['runtime'].loc[new_data['runtime']==0]=1
new_data['time_budget']=new_data['budget'].fillna(0)/new_data['runtime'].fillna(1)

后面我会把剩下的几个特征分析完并且构造合适的特征。

In [ ]:
new_data=new_data.drop('production_companies',axis=1)

In [ ]:
new_data=new_data.drop('crew',axis=1)

In [ ]:
new_data.head()

In [ ]:
new_data.shape
list1=['Avengers','Furious','Beauty and the Beast','ransformers','Rises','Pirates of the Caribbean','Finding Dory','Zootopia','Alice in Wonderland']

In [ ]:
for i in list1:
    new_data['title_is'+i]=new_data['title'].fillna('').apply(lambda x: 1 if i in x else 0)

In [ ]:
for col in ['tagline', 'overview','title']:
    new_data['len_' + col] =new_data[col].fillna('').apply(lambda x: len(x))
    new_data['words_' + col] = new_data[col].fillna('').apply(lambda x: len(x.split(' ')))
    new_data=new_data.drop(col,axis=1)

In [ ]:
new_data.head()

In [ ]:
# new_data['budget'].loc[new_data['budget']==0]=1
# new_data['popularity'].loc[new_data['popularity']==0]=1
# new_data['budget'] = np.log1p(new_data['budget'])
# new_data['popularity'] = np.log1p(new_data['popularity'])
new_data['budget_popularity']=new_data['budget']*1.0/new_data['popularity']
new_data['budget_popularity2']=new_data['budget']*1.0/new_data['popularity2']

In [ ]:
a=new_data.groupby(['release_year']).release_month.value_counts()
b=new_data.groupby(['release_year','release_month']).release_day.value_counts()
new_data[(new_data['release_year']==2017)&(new_data['release_day']==30)]


In [ ]:
new_data['_releaseYear_popularity_ratio'] = new_data['release_year'] / new_data['popularity']
new_data['_releaseYear_popularity_ratio21'] = new_data['release_year'] / new_data['popularity2']
new_data['_releaseYear_popularity_ratio2'] = new_data['popularity'] / new_data['release_year']
new_data['_releaseYear_popularity_ratio22'] = new_data['popularity2'] / new_data['release_year']
    
new_data['runtime_to_mean_year'] = new_data['runtime'] / new_data.groupby("release_year")["runtime"].transform('mean')
new_data['popularity_to_mean_year'] = new_data['popularity'] / new_data.groupby("release_year")["popularity"].transform('mean')
new_data['popularity_to_mean_year'] = new_data['popularity'] / new_data.groupby("release_year")["popularity2"].transform('mean')
new_data['budget_to_mean_year'] = new_data['budget'] / new_data.groupby("release_year")["budget"].transform('mean')



In [ ]:
# new_data['year_day_count']=0
# new_data['year_month_count']=0
# for i in range(len(a)):
#     new_data['year_month_count'][(new_data['release_year']==a.index[i][0])&(new_data['release_month']==a.index[i][1])]=a.values[i]

In [ ]:
# for i in range(len(b)):
#     new_data['year_day_count'][(new_data['release_year']==b.index[i][0])&(new_data['release_month']==b.index[i][1])&(new_data['release_day']==b.index[i][2])]=b.values[i]

In [ ]:
new_train=new_data.loc[train.index]
new_test=new_data.loc[test.index]

In [ ]:
new_test.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
x=new_train.drop('revenue',axis=1)
y=new_train['revenue']
x=x.fillna(0)
from sklearn.tree import DecisionTreeRegressor

# Create Decision Tree with max_depth = 6
decision_tree = DecisionTreeRegressor(max_depth = 6)
decision_tree.fit(x, y)


# clf2 = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
# scores2 = cross_val_score(clf2,x, y)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
import xgboost
model_XG = xgboost.XGBRegressor() 
model_XG.fit(x_train,y_train)
y_predict_rf = model_XG.predict(x_test)
print(mean_squared_error(y_test, y_predict_rf))


In [ ]:
X_test=new_test.drop('revenue',axis=1)
X_test=X_test.fillna(0)

In [ ]:
ypred=model_XG.predict(X_test)

In [ ]:
dtrain = xgb.DMatrix(x, y)
dtest = xgb.DMatrix(X_test)

In [ ]:
xgb_params = {
    'eta': 0.08,
    'max_depth': 15,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=400, verbose_eval=50, show_stdv=False)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)
fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [ ]:
y_predict1 = abs(model.predict(dtest))

In [ ]:
train=x
y_train=np.log1p(y)
test=X_test
y=y_train

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values,y_train.values, scoring="neg_mean_squared_error", cv=kf))
    return(rmse)

def eval_model(model, name):
    start_time = time.time()
    score = rmsle_cv(model)
    print("{} score: {:.4f} ({:.4f}),     execution time: {:.1f}".format(name, score.mean(), score.std(), time.time()-start_time))

In [ ]:
mod_lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.005, random_state=1))
eval_model(mod_lasso, "lasso")
mod_enet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
eval_model(mod_enet, "enet")
mod_cat = CatBoostRegressor(iterations=10000, learning_rate=0.01,
                            depth=10, eval_metric='RMSE',
                            colsample_bylevel=0.7, random_seed = 17, silent=True,
                            bagging_temperature = 0.2, early_stopping_rounds=200)
eval_model(mod_cat, "cat")
mod_gboost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=10, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state=5)
eval_model(mod_gboost, "gboost")
mod_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=10, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state=7, nthread=-1)
eval_model(mod_xgb, "xgb")
mod_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=255,
                              learning_rate=0.05, n_estimators=650,
                              max_bin=58, bagging_fraction=0.80,max_depth=8, 
                              bagging_freq=5, feature_fraction=0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf=7, min_sum_hessian_in_leaf=11)
eval_model(mod_lgb, "lgb")

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    
mod_stacked = StackingAveragedModels(base_models = (mod_cat, mod_xgb, mod_gboost, mod_lgb), meta_model = mod_lasso)
eval_model(mod_stacked, "stacked")

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def predict(model):
    model.fit(train.values,y_train.values)
    train_pred = model.predict(train.values)
    pred = np.expm1(model.predict(test.values))
    print(rmsle(y_train, train_pred))
    return (pred)

In [ ]:
prediction = predict(mod_lasso)
prediction = predict(mod_enet)
prediction = predict(mod_xgb)
prediction = predict(mod_gboost)
prediction = predict(mod_lgb)
prediction = predict(mod_stacked)
prediction

In [ ]:
# y_pre=(abs(ypred+y_predict))/2
new_test['id']=new_test.index
new_test['revenue']=prediction
new_test[['id','revenue']].to_csv('submission_Dragon2.csv', index=False)
new_test[['id','revenue']].head()

由于该项目的预测值非常大,可能需要模型堆叠,所以这里只是拿一个简单的模型跑一下,每天写这个的时间有限